In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *



global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')

batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [30, 70]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [88]:
print(model.features[23])
print(model.features[24])
print(model.features[25])
print(model.features[26])
print(model.features[27])

print(model.features[28])


MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
QuantConv2d(
  256, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace=True)
QuantConv2d(
  8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
ReLU(inplace=True)


In [ ]:

lr = 6e-4
weight_decay = 6e-3
epochs = 300
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

fdir = 'result/'+'VGG_16_project'

        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 1.042 (1.042)	Data 0.999 (0.999)	Loss 2.4765 (2.4765)	Prec 10.156% (10.156%)
Epoch: [0][100/391]	Time 0.038 (0.051)	Data 0.002 (0.012)	Loss 1.6814 (1.8940)	Prec 41.406% (29.339%)
Epoch: [0][200/391]	Time 0.036 (0.046)	Data 0.003 (0.007)	Loss 1.5946 (1.7513)	Prec 43.750% (34.853%)
Epoch: [0][300/391]	Time 0.041 (0.045)	Data 0.003 (0.006)	Loss 1.3900 (1.6728)	Prec 46.875% (37.749%)
Validation starts
Test: [0/79]	Time 0.516 (0.516)	Loss 1.3822 (1.3822)	Prec 46.875% (46.875%)
 * Prec 49.010% 
best acc: 49.010000
Epoch: [1][0/391]	Time 1.050 (1.050)	Data 0.987 (0.987)	Loss 1.2585 (1.2585)	Prec 56.250% (56.250%)
Epoch: [1][100/391]	Time 0.036 (0.052)	Data 0.002 (0.012)	Loss 1.3005 (1.3809)	Prec 49.219% (49.373%)
Epoch: [1][200/391]	Time 0.043 (0.047)	Data 0.002 (0.007)	Loss 1.1016 (1.3547)	Prec 61.719% (50.641%)
Epoch: [1][300/391]	Time 0.043 (0.045)	Data 0.002 (0.006)	Loss 1.2513 (1.3273)	Prec 55.469% (51.806%)
Validation starts
Test: [0/79]	Time 0.721 (0.721)	Loss 1.

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum


In [5]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda" if use_gpu else "cpu") 

model()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [3]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i=0
for layer in model.modules():
    i = i+1
    if ( i == 38):
        print(layer)
        layer.register_forward_pre_hook(save_output) 
dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to(device)
labels = labels.to(device) 
out = model(images)

QuantConv2d(
  8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


In [4]:
print(model.features[27])

QuantConv2d(
  8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


In [5]:
w_bits = 4
weight_q = model.features[27].weight_q # quantized value is stored during the training
w_alpha =  model.features[27].weight_quant.wgt_alpha  # alpha is defined in your model already. bring it out here
w_delta =  w_alpha/(2**(w_bits-1)-1)  # delta can be calculated by using alpha and w_bit
weight_int = weight_q/w_delta # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 7.,  7.,  7.],
          [ 7.,  7.,  7.],
          [-7., -7., -7.]],

         [[ 7., -7., -7.],
          [ 7.,  7., -7.],
          [-7., -7., -7.]],

         [[ 7., -7.,  7.],
          [ 7., -7.,  7.],
          [ 7.,  7.,  7.]],

         [[-7., -7., -7.],
          [ 7., -7.,  7.],
          [ 7., -7., -7.]],

         [[ 7.,  7., -7.],
          [ 7.,  7., -7.],
          [-7., -7., -7.]],

         [[ 7.,  7., -7.],
          [-7., -7.,  7.],
          [-7., -7.,  7.]],

         [[-7., -7., -7.],
          [-7., -7., -7.],
          [ 7., -7.,  7.]],

         [[ 7.,  7.,  7.],
          [ 7.,  7., -7.],
          [ 7.,  7., -7.]]],


        [[[-7.,  7., -7.],
          [ 7.,  7., -7.],
          [ 7., -7., -7.]],

         [[ 7.,  7.,  7.],
          [ 7.,  7., -7.],
          [ 7.,  7.,  7.]],

         [[-7.,  7.,  7.],
          [-7.,  7.,  7.],
          [-7., -7., -7.]],

         [[-7., -7., -7.],
          [-7., -7., -7.],
          [ 7.,  7.,  7.]],

  

In [7]:
x_bits = 4    
x = save_output.outputs[0][0]  # input of the 2nd conv layer
x_alpha  = model.features[27].act_alpha.item()
x_delta = x_alpha/(2**(x_bits)-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x
x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 4.0000, 11.0000,  9.0000,  4.0000],
          [ 2.0000,  9.0000,  0.0000,  0.0000],
          [ 6.0000, 15.0000,  8.0000,  0.0000],
          [ 1.0000,  4.0000,  3.0000,  0.0000]],

         [[ 0.0000,  0.0000,  1.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 3.0000,  5.0000,  1.0000,  1.0000],
          [ 3.0000,  4.0000,  5.0000,  3.0000]],

         [[ 0.0000,  0.0000,  3.0000,  7.0000],
          [ 0.0000,  0.0000,  3.0000,  6.0000],
          [ 0.0000,  0.0000,  0.0000,  1.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         ...,

         [[ 2.0000,  3.0000,  3.0000,  0.0000],
          [ 0.0000,  0.0000,  2.0000,  0.0000],
          [ 0.0000,  8.0000, 15.0000,  8.0000],
          [ 0.0000,  2.0000,  6.0000,  3.0000]],

         [[ 0.0000,  0.0000,  0.0000,  1.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  4.0000,  2.0000,  0.0000],
          [ 0.0000,  4.0000,  4.0000,  3.0000]],

         [[ 0.0

In [8]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_recovered = output_int * x_delta * w_delta # recover with x_delta and w_delta
print(output_recovered)

tensor([[[[ 1.0969e-02,  1.0421e-01],
          [ 5.4845e-02, -5.4845e-02]],

         [[-8.7752e-02,  1.3711e-01],
          [ 1.0969e-02,  8.7752e-02]],

         [[-9.8721e-02, -2.3583e-01],
          [-2.1938e-02, -8.7752e-02]],

         ...,

         [[ 2.9616e-01,  4.5521e-01],
          [ 2.5229e-01,  5.3748e-01]],

         [[-3.2907e-02, -2.5777e-01],
          [ 1.0969e-01,  7.6783e-02]],

         [[-1.0969e-02,  2.7422e-02],
          [-4.3876e-02, -2.6326e-01]]],


        [[[-4.3876e-02, -1.0969e-02],
          [-4.3876e-02,  2.8519e-01]],

         [[ 3.2907e-02,  2.3035e-01],
          [-3.8391e-01,  3.2907e-02]],

         [[ 5.1554e-01,  1.2066e-01],
          [ 3.7295e-01,  3.0713e-01]],

         ...,

         [[ 4.0585e-01,  2.4132e-01],
          [ 4.6070e-01,  1.6453e-01]],

         [[-2.1938e-01,  7.6783e-02],
          [ 1.9744e-01,  5.5942e-01]],

         [[-1.0969e-02, -1.3163e-01],
          [ 3.5101e-01,  2.3035e-01]]],


        [[[ 3.7843e-01,  5.210

In [9]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, bias = False)
weight = model.features[27].weight_q

conv_ref.weight = weight
conv_ref.bias = model.features[3].bias

output_ref = conv_ref(x)
print(abs((output_ref - output_recovered)).mean())


tensor(0.0281, device='cuda:0', grad_fn=<MeanBackward0>)


In [10]:

a_int = x_int[0,:,:,:]  # pick only one input out of batch
# a_int.size() = [8, 4, 4]
print(a_int.size())
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij
# w_int.weight.size() = torch.Size([8, 8, 9])
print(w_int.size())
padding = 1
stride = 1
array_size = 8 # row and column number

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group
print(nig)
print(njg)
icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel
print(icg)
print(ocg)
ic_tileg = range(int(len(icg)/array_size))
oc_tileg = range(int(len(ocg)/array_size))
#print(ic_tileg)
#print(oc_tileg)
kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size
print(kijg)

######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(nig)+padding*2).cuda()
# a_pad.size() = [8, 4+2pad, 4+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
# a_pad.size() = [8, (4+2pad)*(4+2pad)]




torch.Size([8, 4, 4])
torch.Size([8, 8, 9])
range(0, 4)
range(0, 4)
range(0, 8)
range(0, 8)
range(0, 9)


In [11]:

p_nijg = range(a_pad.size(1)) ## paded activation's nij group [0, ...34*34-1]
print(p_nijg)
psum = torch.zeros( array_size, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:       
    for nij in p_nijg:     # time domain, sequentially given input
        m = nn.Linear(array_size, array_size, bias=False)
        m.weight = torch.nn.Parameter(w_int[:,:,kij])
        psum[:, nij, kij] = m(a_pad[:,nij]).cuda()
 

range(0, 36)


In [12]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32 + 2*pad = 34

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1) #34 - 2 - 1 + 1 = 32
o_nijg = range(o_ni_dim**2) # [0, 32*32-1]    
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:  #[0, ... 8]
        out[:,o_nij] = out[:,o_nij] + \
        psum[:, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 2nd index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

In [38]:
### show this cell partially. The following cells should be printed by students ###
nij = 0 # just a random number
X = a_pad[:,nij:nij+8]  # [tile_num, array row num, time_steps]
print(X)
bit_precision = 4
file = open('activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(round(X[7-j,i].item()))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.0000, 11.0000,  9.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.0000,  3.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.0000,  3.0000,  3.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.0000,  1.0000]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [32]:
### Complete this cell ###
kij = 8
W = w_int[:,:,kij]  # w_tile[tile_num, array col num, array row num, kij]


bit_precision = 4
file = open('weight8.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')


for i in range(W.size(1)):  # time step
    for j in range(W.size(0)):  # row #
        value = round(W[7-j, i].item())  # Get the value
        if value < 0:
            value = (1 << bit_precision) + value  # Two's complement for negatives
        W_bin = '{0:04b}'.format(value)  # Format as 4-bit binary
        for k in range(bit_precision):
            file.write(W_bin[k])        
    file.write('\n')
file.close()

In [37]:
### Complete this cell ###
kij = 0
nij = 2

psum_tile = psum[:,nij:nij+8,kij]  
# psum[len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)]
bit_precision = 16
file = open('psum.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(psum_tile.size(1)):  # time step
    for j in range(psum_tile.size(0)):  # row #
        value = round(psum_tile[7-j, i].item())  # Get the value
        if value < 0:
            value = (1 << bit_precision) + value  # Two's complement for negatives
        psum_bin = '{0:016b}'.format(value)  # Format as 4-bit binary
        for k in range(bit_precision):
            file.write(psum_bin[k])        
    file.write('\n')
file.close()
